# MLP

> Multilayer perceptron model

In [ ]:
#| default_exp models.mlp

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2
from nbdev.showdoc import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
#| export
import torch.nn as nn
import torch

from pytorch_lightning import LightningModule
from pytorch_lightning.utilities import rank_zero_only
from torchmetrics import Accuracy
from hydra.utils import instantiate
from omegaconf import OmegaConf

from nimrod.data.datasets import MNISTDataModule

## Model

In [ ]:
#| export
class MLP(nn.Module):
    def __init__(self, n_in=32*32*3, n_h=64, n_out=10):
        super().__init__()
        l1 = nn.Linear(n_in, n_h)
        l2 = nn.Linear(n_h, n_out)
        relu = nn.ReLU()
        self.layers = nn.Sequential(l1,l2,relu)
        
    def forward(self, x):
        return self.layers(x)

### Usage

In [ ]:
image = torch.rand((5,28*28))
mlp = MLP(n_in=28*28, n_h=64,n_out=10)
out = mlp(image)
print(out.shape)

torch.Size([5, 10])


## Module

In [ ]:
#| export
class MLP_PL(LightningModule):
    def __init__(self, mlp:MLP):
        super().__init__()
        self.save_hyperparameters(ignore=['mlp'])
        self.mlp = mlp
        self.loss = nn.CrossEntropyLoss()
        self.accuracy = Accuracy(task="multiclass", num_classes=10)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer
    
    def forward(self, x):
        return(self.mlp(x))

    def training_step(self, batch, batch_idx):
        x, y = batch
        x = x.view(x.size(0), -1)
        y_hat = self.mlp(x)
        loss = self.loss(y_hat, y)
        return loss
    
    def _step(self, batch, batch_idx):
        x, y = batch
        x = x.view(x.size(0), -1)
        y_hat = self.mlp(x)
        loss = self.loss(y_hat, y)
        acc = self.accuracy(y_hat, y)
        return loss, acc
    
    def validation_step(self, batch, batch_idx, prog_bar=True, on_step=False, on_epoch=True, sync_dist=True):
        loss, acc = self._step(batch, batch_idx)
        metrics = {"val/loss":loss, "val/acc": acc}
        self.log_dict(metrics, on_step=on_step, on_epoch=on_epoch, sync_dist=sync_dist)
    
    def test_step(self, batch, batch_idx, prog_bar=True, on_step=False, on_epoch=True, sync_dist=True):
        loss, acc = self._step(batch, batch_idx)
        metrics = {"test/loss":loss, "test/acc": acc}
        self.log_dict(metrics, on_step=on_step, on_epoch=on_epoch, sync_dist=sync_dist)

    def predict_step(self, batch, batch_idx, dataloader_idx=0):
        x, y = batch
        y_hat = self.mlp(x)
        return y_hat


### Usage

In [ ]:
mlp_pl = MLP_PL(mlp)
b = torch.rand((5,28*28))
y = mlp_pl(b)
print(y.shape)

torch.Size([5, 10])


### Training
c.f. recipes/image/mnist

In [ ]:
#| hide
cfg = OmegaConf.load('../conf/datamodules/mnist.yaml')
datamodule = instantiate(cfg)
datamodule.prepare_data()
datamodule.setup()
x = datamodule.data_test[0][0]
label = datamodule.data_test[0][1]
print(x.view(x.size(0), -1).shape)

torch.Size([1, 784])


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()